In [ ]:
# both can change
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import json
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K
from keras.utils.vis_utils import plot_model
from scipy.interpolate import LinearNDInterpolator, interpn
from scipy.optimize import root

# Local modules for handling data and running moisture models
import data_funcs as datf
from data_funcs import format_raws, retrieve_raws, format_precip, fixnan
import moisture_models as mod
from moisture_rnn import create_RNN_2, staircase, create_rnn_data, train_rnn, rnn_predict
from data_funcs import raws_data, synthetic_data, plot_data, check_data, mse_data

meso_token="b40cb52cbdef43ef81329b84e8fd874f"


In [ ]:
def vprint(*args):
    if verbose: 
        for s in args[:(len(args)-1)]:
            print(s, end=' ')
        print(args[-1])

## Retrieve RAWS Data

In [ ]:
raws_dat=raws_data(start='201806010800', hours=1200, h2=300, stid="BKCU1",meso_token=meso_token)
raws_dat_orig = raws_dat

check_data(raws_dat)
%matplotlib inline
plot_data(raws_dat)

## Retrieve RTMA Function

<mark>Not needed?</mark>

## Interface
Jonathon changes above  create each case as a dictionary, then dictionary of dictionaries, figure out how to store and load dictionaries as a file. json is possible but: cannot contain datetime objects
look into pickle also compresses while json is plain text clone wrfxpy look how for idioms, pickle added jan/angel lager
Jan will edit from here below. 
cases will be extracted from dictionary as global variables for now at least

In [ ]:
np.random.seed(seed=123)

In [ ]:
# dictionary raws_dat has all that is needed for the run 
# keeping the name raws_dat for now even if it may not be raws data

    

In [ ]:
synt_dat=synthetic_data()
%matplotlib inline
plot_data(synt_dat)

## Fit Augmented KF

In [ ]:
m,Ec = mod.run_augmented_kf(raws_dat)  # extract from state
raws_dat['m']=m
plot_data(raws_dat,title2='augmented KF')

In [ ]:
plot_data(raws_dat,hmin=900,hmax=1200,title2='augmented KF prediction detail')

In [ ]:
mse_data(raws_dat)


## Fit RNN Model

In [ ]:
# Set seed for reproducibility
tf.random.set_seed(123)

In [ ]:
verbose = False
# rnn_dat = create_rnn_data(raws_dat,scale=False, hours=None, h2=None)
rnn_dat = json.load(open('rnn_orig.json','r'))

In [ ]:
model_predict = train_rnn(
    rnn_dat,
    rnn_dat['hours'],
    activation=['linear','linear'],
    hidden_units=3,
    dense_units=1,
    dense_layers=1,
    verbose = verbose
)

In [ ]:
verbose = 0
raws_dat['m'] = rnn_predict(model_predict, rnn_dat, raws_dat['hours'])

In [ ]:
plot_data(raws_dat,title2='with trained RNN')


In [ ]:
plot_data(raws_dat,title2='RNN training',hmin=0,hmax=300)

In [ ]:
plot_data(raws_dat,title2='RNN prediction',hmin=300,hmax=600)

In [ ]:
plot_data(raws_dat,title2='RNN prediction',hmin=600,hmax=900)

In [ ]:
plot_data(raws_dat,hmin=900,hmax=1200,title2='RNN prediction detail')

In [ ]:
mse_data(raws_dat)